# Riverscapes Exploration Notebook

This notebook is configured to allow interactive analysis of Riverscapes data.
It sets up the python path to include the `src` directory, enabling imports from `reports` and `util`.


In [ ]:
import sys
import os
from pathlib import Path

# Add the src directory to the path so we can import from reports and util
# This assumes the notebook is running from scripts/exploration/
# Adjust relative path if moved
current_dir = Path.cwd().resolve()
# We want to go up two levels from scripts/exploration to root
project_root = current_dir.parent.parent

src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"Project root added to path: {project_root}")


Project root added to path: C:\nardata\localcode\rs-reports-gen


In [26]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

# Import from src
# These imports will work once the path is set up above
from util.athena import query_to_dataframe, get_field_metadata
# this is only needed if the util code has changed in middle of our session
# import importlib
# import util.pandas.pandas_utilities
# importlib.reload(util.pandas.pandas_utilities)
from util.pandas.pandas_utilities import pprint_df_meta
from util.pandas import RSFieldMeta

# Set pandas options for better display
pd.set_option('display.max_columns', None)


In [4]:
# Define the query (from notebook.py)
qry = """
SELECT r.seg_distance, r.fcode, rm.* 
FROM rsdynamics r 
join rsdynamics_metrics rm on (r.rd_project_id = rm.rd_project_id and r.dgo_id = rm.dgo_id)
where r.huc='364256' and level_path = '364256000000001'
"""

# Load the dataframe
print("Querying Athena...")
# Note: Ensure you have AWS credentials configured in your environment
try:
    df = query_to_dataframe(qry, "dynamicsmetrics")
    print(f"Dataframe loaded with shape: {df.shape}")
    display(df.head())
except Exception as e:
    print(f"Error executing query: {e}")
    print("Ensure AWS credentials are set and 'util.athena' is reachable.")


Querying Athena...
[DEBUG] [Athena unload query to DF] Query dynamicsmetrics:
              
              SELECT r.seg_distance, r.fcode, rm.* 
              FROM rsdynamics r 
              join rsdynamics_metrics rm on (r.rd_project_id = rm.rd_project_id and r.dgo_id = rm.dgo_id)
              where r.huc='364256' and level_path = '364256000000001'
              
[DEBUG] [Athena unload query to DF] Query dynamicsmetrics to dataframe completed.
Dataframe loaded with shape: (24832, 13)


,seg_distance,fcode,dgo_id,landcover,epoch_length,epoch_name,confidence,area,areapc,width,widthpc,huc,rd_project_id
0,<NA>,<NA>,3137,wet,5,2024_2024,95,0.0,NaN,NaN,NaN,364256,f5a45736-3a80-437b-88eb-ccd774e3b6d1
1,<NA>,<NA>,3137,wet,5,2024_2024,68,0.0,NaN,NaN,NaN,364256,f5a45736-3a80-437b-88eb-ccd774e3b6d1
2,<NA>,<NA>,3137,wet,5,2019_2024,95,0.0,NaN,NaN,NaN,364256,f5a45736-3a80-437b-88eb-ccd774e3b6d1
3,<NA>,<NA>,3137,wet,5,2019_2024,68,0.0,NaN,NaN,NaN,364256,f5a45736-3a80-437b-88eb-ccd774e3b6d1
4,<NA>,<NA>,3137,wet,5,2014_2019,95,0.0,NaN,NaN,NaN,364256,f5a45736-3a80-437b-88eb-ccd774e3b6d1


In [5]:
# Interactive Visualization
if 'df' in locals() and not df.empty:
    # Example: Histogram of segment distance
    if 'seg_distance' in df.columns:
        fig = px.histogram(df, x="seg_distance", title="Distribution of Segment Distances")
        fig.show()
    
    # You can add more plots here using px.scatter, px.line, etc.
else:
    print("Dataframe not available for plotting.")


In [6]:
# Convert columns to categorical
cols_to_convert = ['confidence', 'landcover', 'fcode', 'epoch_length']
for col in cols_to_convert:
    if col in df.columns:
        df[col] = df[col].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24832 entries, 0 to 24831
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   seg_distance   24160 non-null  Int64   
 1   fcode          23904 non-null  category
 2   dgo_id         24832 non-null  Int64   
 3   landcover      24832 non-null  category
 4   epoch_length   24832 non-null  category
 5   epoch_name     24832 non-null  string  
 6   confidence     24832 non-null  category
 7   area           24832 non-null  float64 
 8   areapc         6640 non-null   float64 
 9   width          24256 non-null  float64 
 10  widthpc        6640 non-null   float64 
 11  huc            24832 non-null  string  
 12  rd_project_id  24832 non-null  string  
dtypes: Int64(2), category(4), float64(4), string(3)
memory usage: 1.8 MB


In [ ]:
# /Export DataFrame to Excel
from pathlib import Path
output_dir = Path(r'C:\nardata\pydataroot\rpt-riverscapes-dynamics')
xl_output_file = output_dir / 'sample.xlsx'
df.to_excel(xl_output_file, index=False)
print(f'Exported to {xl_output_file}')

Exported to C:\nardata\pydataroot\rpt-riverscapes-dynamics\sample.xlsx


In [17]:
import plotly.express as px
# Filter data where epoch_length is 5 (including all confidence levels)
filtered_df = df[(df['epoch_length'] == '5')]

# Summarize sum of area by landcover, epoch_name, and confidence
summary_df = filtered_df.groupby(['landcover', 'epoch_name', 'confidence'], observed=False)['area'].sum().reset_index()

# Sort by epoch_name
summary_df = summary_df.sort_values('epoch_name')

In [18]:
# Create a line chart with different styles for confidence
fig = px.line(summary_df, x='epoch_name', y='area', color='landcover', line_dash='confidence',
              title='Area by Landcover over Time (Epoch Length 5)',
              line_dash_map={'95': 'solid', '68': 'dash'})
fig.show()

In [19]:
# Loop through metrics and create plots
metrics = ['area', 'areapc', 'width', 'widthpc']
for metric in metrics:
    if metric in df.columns:
        # Group by landcover, epoch_name, confidence
        metric_summary = filtered_df.groupby(['landcover', 'epoch_name', 'confidence'], observed=False)[metric].sum().reset_index()
        metric_summary = metric_summary.sort_values('epoch_name')
        
        # Create line chart
        fig = px.line(metric_summary, x='epoch_name', y=metric, color='landcover', line_dash='confidence',
                      title=f'{metric.capitalize()} by Landcover over Time (Epoch Length 5)',
                      line_dash_map={'95': 'solid', '68': 'dash'})
        fig.show()

In [29]:
pprint_df_meta(df)

DataFrame: Unnamed
------------------------------------------------------------------------------------------------------------------------
Shape (rows, cols): (24832, 13)

[ERROR] [RSFieldMeta] Ambiguous column 'dgo_id'. Found in tables: RSDynamics DGOs, RSDynamics Metrics. Provide table_name.
[ERROR] [RSFieldMeta] Ambiguous column 'dgo_id'. Found in tables: RSDynamics DGOs, RSDynamics Metrics. Provide table_name.
[ERROR] [RSFieldMeta] Ambiguous column 'huc'. Found in tables: RSDynamics DGOs, RSDynamics Metrics. Provide table_name.
[ERROR] [RSFieldMeta] Ambiguous column 'huc'. Found in tables: RSDynamics DGOs, RSDynamics Metrics. Provide table_name.
[ERROR] [RSFieldMeta] Ambiguous column 'rd_project_id'. Found in tables: RSDynamics DGOs, RSDynamics Metrics. Provide table_name.
[ERROR] [RSFieldMeta] Ambiguous column 'rd_project_id'. Found in tables: RSDynamics DGOs, RSDynamics Metrics. Provide table_name.
METADATA SUMMARY
#    Column                                   Type              

In [28]:
# Generate Report Artifacts / Metadata Exploration
# This section demonstrates using other parts of src/reports or util

if 'df' in locals():
    # Example: Check field metadata using util function
    try:
        _FIELD_META = RSFieldMeta()  # Instantiate the Borg singleton. We can reference it with this object or RSFieldMeta()
        metadf = get_field_metadata(authority='data-exchange-scripts', authority_name='rsdynamics_to_athena', layer_id=["rsdynamics","rsdynamics_metrics"])
        _FIELD_META.field_meta = metadf
        print("\nField Metadata:")
        # Display first few keys
        keys = list(metadf.keys())[:5]
        for k in keys:
            print(f"{k}: {metadf[k]}")
    except Exception as e:
        print(f"Could not retrieve metadata: {e}")


[INFO] [Get metadata] Getting metadata from Athena
[DEBUG] [Athena unload query to DF] Query :
              
              SELECT layer_id, layer_name AS table_name, name, friendly_name, data_unit, description, theme, dtype
              FROM layer_definitions_latest
              WHERE authority = 'data-exchange-scripts'  AND authority_name IN ('rsdynamics_to_athena')  AND layer_id IN ('rsdynamics', 'rsdynamics_metrics') 
              
[DEBUG] [Athena unload query to DF] Query  to dataframe completed.
[INFO] [RSFieldMeta] Setting metadata for the first time.

Field Metadata:
layer_id: 0             rsdynamics
1             rsdynamics
2             rsdynamics
3             rsdynamics
4             rsdynamics
5             rsdynamics
6             rsdynamics
7             rsdynamics
8             rsdynamics
9             rsdynamics
10            rsdynamics
11            rsdynamics
12            rsdynamics
13            rsdynamics
14            rsdynamics
15            rsdynamics
16   